In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

In [34]:
url= 'https://www.imdb.com/search/title/?num_votes=100000,&sort=user_rating,desc&title_type=tv_series&'
data=requests.get(url)
news_data = []
soup = BeautifulSoup(data.content, 'html.parser')

# Creating database

In [102]:
name=[]
certi=[]#null
rate=[]
run=[]
genre=[]
gross=[]#null
director=[]
summary=[]


movie_Data=soup.find_all('div',attrs = {'class':'lister-item-content'})

for data in movie_Data:
    name.append(data.h3.a.text)
    run.append(data.find('span', attrs = {'class':'runtime'}).text)
    rating=data.find('div', attrs = {'class':'inline-block ratings-imdb-rating'})
    rate.append(rating['data-value'])
    genre.append(data.find('span', attrs = {'class':'genre'}).text.replace('\n', ' '))
    temp=data.find('p',attrs = {'class':''})
    director.append(temp.a.text)    
    temp1=data.find_all('p',{'class':'text-muted'})
    temp1=temp1[-1].text
    summary.append(str(temp1.replace('\n','').strip()))
    
df = pd.DataFrame({'Movie_Name':name,'rating':rate,'runtime':run,'genre':genre,'Director':director,'Summary':summary})
df.head()

df.to_csv('preview.csv', index=False)  

# Pre-processing
Expanding Contraction,
Remove Special char

In [98]:
import re
from contractions import CONTRACTION_MAP

contractions_re = re.compile('(%s)' % '|'.join(CONTRACTION_MAP.keys()))

def expand_contractions(s, contractions_dict=CONTRACTION_MAP):
    def replace(match):
            return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)


text=("A gangster family epic set in 1919 Birmingham, England; centered on a gang who sew razor blades in the peaks of their caps, and their fierce boss Tommy Shelby.").lower()
expand_contractions(text)


'a gangster family epic set in 1919 birmingham, england; centered on a gang who sew razor blades in the peaks of their caps, and their fierce boss tommy shelby.'

In [106]:

def remove_special_characters(text):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', text)
    return text

remove_special_characters("A gangster family epic set in 1919 Birmingham, England; centered on a gang who sew razor blades in the peaks of their caps, and their fierce boss Tommy Shelby.")

'A gangster family epic set in 1919 Birmingham England centered on a gang who sew razor blades in the peaks of their caps and their fierce boss Tommy Shelby'

Stemming and Lemmatization 

In [193]:
#stemming
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

def simple_stemmer(text):
    words = word_tokenize(text) 
    for w in words: 
        #print(w, " : ", PorterStemmer().stem(w)) 
        text=re.sub(w,PorterStemmer().stem(w),text,count=1)
    return text

simple_stemmer("Eat Eating Playing played play playing Stemming stem stemmed rocks corpora better.")

'.at eat play play play play stem stem stem rock corpora better.'

In [206]:
#Lemmatization (takes part of speech(pos) - default noun)
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 
  
print("rocks :", lemmatizer.lemmatize("rocks")) 
print("corpora :", lemmatizer.lemmatize("corpora")) 
  
# a denotes adjective in "pos" 
print("better :", lemmatizer.lemmatize("better", pos ="a")) 
print("playing :", lemmatizer.lemmatize("playing", pos ="v")) 
print("playing :", lemmatizer.lemmatize("playing", pos ="a")) 
def simple_lemmatization(text):
    words = word_tokenize(text) 
    for w in words: 
        #print(w, " : ", PorterStemmer().stem(w)) 
        text=re.sub(w,lemmatizer.lemmatize(w),text,count=2)
    return text

simple_lemmatization("Fishing at swimming at the banks of river rocks corpora better")

rocks : rock
corpora : corpus
better : good
playing : play
playing : playing


'Fishing at swimming at the bank of river rock corpus better'

# Stop words

In [224]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()

stopword_list = nltk.corpus.stopwords.words('english')
#stopword_list.remove('no')
#stopword_list.remove('not')
#stopword_list.remove('this')
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

remove_stopwords("This is a sample sentence, showing off the stop words filtration.")

'sample sentence , showing stop words filtration .'

# Finding part of Speech